# DSG: MultinomialNB

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline, make_union
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score, train_test_split,StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from lib_DSG import ColumnSelector, DenseTransformer


folder = os.getcwd() ; print folder

/home/arda/Documents/youtube/oldscript


# Importing data

In [6]:
pd_train = pd.read_csv(folder + '/data/train_only_text_simple.csv', sep=';', dialect=None, compression='infer', doublequote=True, 
            escapechar='\\', quotechar='"', quoting=0, skipinitialspace=False,
            lineterminator=None, header='infer', index_col=None, names=None,
            prefix=None, skiprows=None, skipfooter=None, skip_footer=0, na_values=None,
            na_fvalues=None, true_values=None, false_values=None, delimiter=None, converters=None,
            dtype=None, usecols=None, engine=None, delim_whitespace=False, as_recarray=False,
            na_filter=True, compact_ints=False, use_unsigned=False, low_memory=False, buffer_lines=None,
            warn_bad_lines=True, error_bad_lines=True, keep_default_na=True, thousands=None, comment=None,
            decimal='.', parse_dates=False, keep_date_col=False, dayfirst=False, date_parser=None,
            memory_map=False, float_precision=None, nrows=None, iterator=False, chunksize=None,
            verbose=False, encoding=None, squeeze=False, mangle_dupe_cols=True, tupleize_cols=False,
            infer_datetime_format=False, skip_blank_lines=True)


pd_test = pd.read_csv(folder + '/data/test_only_text_simple.csv', sep=';', dialect=None, compression='infer', doublequote=True, 
            escapechar='\\', quotechar='"', quoting=0, skipinitialspace=False,
            lineterminator=None, header='infer', index_col=None, names=None,
            prefix=None, skiprows=None, skipfooter=None, skip_footer=0, na_values=None,
            na_fvalues=None, true_values=None, false_values=None, delimiter=None, converters=None,
            dtype=None, usecols=None, engine=None, delim_whitespace=False, as_recarray=False,
            na_filter=True, compact_ints=False, use_unsigned=False, low_memory=False, buffer_lines=None,
            warn_bad_lines=True, error_bad_lines=True, keep_default_na=True, thousands=None, comment=None,
            decimal='.', parse_dates=False, keep_date_col=False, dayfirst=False, date_parser=None,
            memory_map=False, float_precision=None, nrows=None, iterator=False, chunksize=None,
            verbose=False, encoding=None, squeeze=False, mangle_dupe_cols=True, tupleize_cols=False,
            infer_datetime_format=False, skip_blank_lines=True)

IOError: File /home/arda/Documents/youtube/oldscript/data/train_only_text_simple.csv does not exist

# First model

In [219]:
dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }
tfv = TfidfVectorizer(lowercase=False, stop_words=None, token_pattern=dico_pattern["match_word_punct"], 
                      ngram_range=(1, 2), max_df=1.0, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=True)

clf = MultinomialNB(alpha=0.05, fit_prior=True, class_prior=None)

pipeline = make_pipeline(ColumnSelector(key='Sentence'), tfv)

In [220]:
Y = pd_train['Author'].values
X = pipeline.fit_transform(pd_train) ; print X.shape

(28723, 70877)


In [221]:
alphas = np.arange(0.01, 0.2, 0.03)
#alphas = [0.06]
print alphas

[ 0.01  0.04  0.07  0.1   0.13  0.16  0.19]


In [223]:
results=[]
for i in alphas:
    clf.alpha = i
    
    skf = StratifiedKFold(Y, n_folds=10, indices=None, shuffle=True, random_state=None)
    scores_skf = cross_val_score(clf, X, Y,scoring='accuracy',cv=skf, n_jobs=-1)

    sss = StratifiedShuffleSplit(Y, 10, test_size=0.2, random_state=0)
    scores_sss = cross_val_score(clf, X, Y,scoring='accuracy',cv=sss, n_jobs=-1)

    print ("SLF: acc: %0.4f, std: %0.4f, SSS: acc: %0.4f, std: %0.4f, alpha: %s" %
           (scores_skf.mean(), scores_skf.std(), scores_sss.mean(), scores_sss.std(), i))

SLF: acc: 0.8193, std: 0.0057, SSS: acc: 0.8155, std: 0.0067, alpha: 0.01
SLF: acc: 0.8245, std: 0.0064, SSS: acc: 0.8197, std: 0.0049, alpha: 0.04
SLF: acc: 0.8231, std: 0.0078, SSS: acc: 0.8184, std: 0.0042, alpha: 0.07
SLF: acc: 0.8214, std: 0.0059, SSS: acc: 0.8165, std: 0.0040, alpha: 0.1
SLF: acc: 0.8190, std: 0.0054, SSS: acc: 0.8132, std: 0.0047, alpha: 0.13
SLF: acc: 0.8132, std: 0.0063, SSS: acc: 0.8090, std: 0.0049, alpha: 0.16
SLF: acc: 0.8084, std: 0.0077, SSS: acc: 0.8040, std: 0.0053, alpha: 0.19


In [174]:
clf.C=0.7

skf = StratifiedKFold(Y, n_folds=10, indices=None, shuffle=True, random_state=None)
for train_idx, val_idx in skf:
    x_train, y_train, x_val, y_val = X[train_idx], Y[train_idx], X[val_idx], Y[val_idx]
    
    #x_train = kbest.fit_transform(x_train, y_train)
    #x_val = kbest.transform(x_val)
    
    print clf.fit(x_train, y_train).score(x_val, y_val)

0.807371349096
0.798538622129
0.823181343543
0.801253045597
0.808844011142
0.813022284123
0.804665738162
0.815395332637
0.807665505226
0.812195121951


# Considering more features

In [3]:
dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }

tfv_uni = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_lowercase_only"],
                ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_bi = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_word1"],
                ngram_range=(2, 2), max_df=1.0, min_df=2, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_nnp = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_NNP"],
                ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

tfv_punctuation = TfidfVectorizer(lowercase=False, analyzer=u'word', stop_words=None, token_pattern=dico_pattern["match_punct"],
                ngram_range=(1, 1), max_df=1.0, min_df=1, max_features=None, vocabulary=None,
                binary=True, norm=u'l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

clf = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)

uni = make_pipeline(ColumnSelector(key='Sentence'), tfv_uni)
bi = make_pipeline(ColumnSelector(key='Sentence'), tfv_bi)
nnp = make_pipeline(ColumnSelector(key='Sentence'), tfv_nnp)
punctuation = make_pipeline(ColumnSelector(key='Sentence'), tfv_punctuation)


pipeline = make_union(uni, bi, nnp, punctuation)

In [5]:
Y = pd_train['Author'].values
X = pipeline.fit_transform(pd_train)
X_test = pipeline.transform(pd_test)

print X.shape

 (28723, 77821)


In [6]:
alphas = np.arange(0.01, 0.2, 0.03);
weights=[1]
print alphas

[ 0.01  0.04  0.07  0.1   0.13  0.16  0.19]


In [12]:
for weight in weights:
    print weight
    
    pipeline.transformer_weights=[1.3, 1, 1.1, 1]
    
    tfv_uni.binary=True
    tfv_bi.binary=True
    tfv_nnp.binary=True
    tfv_punctuation.binary=True
    
    X = pipeline.fit_transform(pd_train)
    results=[]
    for i in alphas:
        clf.alpha = i
        
        skf = StratifiedKFold(Y, n_folds=10, indices=None, shuffle=True, random_state=None)
        scores_skf = cross_val_score(clf, X, Y,scoring='accuracy',cv=skf, n_jobs=-1)
        
        sss = StratifiedShuffleSplit(Y, 10, test_size=0.2, random_state=0)
        scores_sss = cross_val_score(clf, X, Y,scoring='accuracy',cv=sss, n_jobs=-1)

        print ("SLF: acc: %0.4f, std: %0.4f, SSS: acc: %0.4f, std: %0.4f, alpha: %s" %
               (scores_skf.mean(), scores_skf.std(), scores_sss.mean(), scores_sss.std(), i))

1
SLF: acc: 0.8202, std: 0.0084, SSS: acc: 0.8181, std: 0.0058, alpha: 0.01
SLF: acc: 0.8267, std: 0.0064, SSS: acc: 0.8246, std: 0.0054, alpha: 0.04
SLF: acc: 0.8277, std: 0.0064, SSS: acc: 0.8242, std: 0.0053, alpha: 0.07
SLF: acc: 0.8251, std: 0.0116, SSS: acc: 0.8231, std: 0.0048, alpha: 0.1
SLF: acc: 0.8263, std: 0.0063, SSS: acc: 0.8216, std: 0.0043, alpha: 0.13
SLF: acc: 0.8244, std: 0.0041, SSS: acc: 0.8196, std: 0.0047, alpha: 0.16
SLF: acc: 0.8233, std: 0.0051, SSS: acc: 0.8178, std: 0.0047, alpha: 0.19


In [ ]:
#Accuracy: 0.7806 (+/- 0.0721), alpha: 0.08 ===>88,28531%
#Accuracy: 0.7834 (+/- 0.0747), alpha: 0.05 ===>86,87608%
#Accuracy: 0.7810 (+/- 0.0739), alpha: 0.04 ===>87,17525%
#Accuracy: 0.7811 (+/- 0.0688), alpha: 0.14 ===>88,02551%.

#SLF: acc: 0.8278, std: 0.0064, SSS: acc: 0.8249, std: 0.0062, alpha: 0.05 ===> 88,23020%

In [13]:
alpha = 0.07

l = []
for i in range(10):
    
    NB= MultinomialNB(alpha=alpha , fit_prior=True, class_prior=None)

    scores=[]
    confusion_mat = np.zeros([6,6],dtype=int)
    for cv in range(5):
        x_train, x_val, y_train, y_val = train_test_split(X,Y, test_size=0.2)
        
        NB.fit(x_train,y_train)
        
        scores.append(NB.score(x_val,y_val))
        confusion_mat+=confusion_matrix(NB.predict(x_val),y_val)
    l.append(np.mean(scores))
    print ("Accuracy: %0.4f (+/- %0.4f), alpha: %s" % (np.mean(scores), np.std(scores) * 2, i))
    
print NB.classes_
print confusion_mat
print ("Accuracy: %0.4f (+/- %0.4f)" % (np.mean(l), np.std(scores) * 2))

Accuracy: 0.8251 (+/- 0.0115), alpha: 0
Accuracy: 0.8236 (+/- 0.0065), alpha: 1
Accuracy: 0.8247 (+/- 0.0084), alpha: 2
Accuracy: 0.8199 (+/- 0.0060), alpha: 3
Accuracy: 0.8283 (+/- 0.0054), alpha: 4
Accuracy: 0.8188 (+/- 0.0051), alpha: 5
Accuracy: 0.8233 (+/- 0.0066), alpha: 6
Accuracy: 0.8256 (+/- 0.0028), alpha: 7
Accuracy: 0.8253 (+/- 0.0100), alpha: 8
Accuracy: 0.8240 (+/- 0.0071), alpha: 9
['austen' 'doyle' 'poe' 'shakespeare' 'twain' 'wilde']
[[8939  517   47   38  219  638]
 [ 246 4219   93   49  320  439]
 [   0    5   50    0    2    1]
 [  62   57   21 1906  208   52]
 [ 124  374   18   63 5159  255]
 [ 447  482   17   37  226 3395]]
Accuracy: 0.8239 (+/- 0.0071)


# SUBMIT KAGGLE

In [44]:
NB.fit(X,Y)

MultinomialNB(alpha=0.04, class_prior=None, fit_prior=True)

In [45]:
y_pred = NB.predict(X_test)

In [46]:
submit = pd.DataFrame(index=None)
submit['Id']=pd_test['Id']
submit['Pred']=y_pred

In [47]:
submit.to_csv(folder+'/results/0.arda1.csv',sep=';',index=None)